In the modular approach I will create separate files for different model types. This one will be for beta_stack_model_sil_credo_score

# Define Library

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
import tempfile
import time
from datetime import datetime
import uuid
import joblib
import uuid
from sklearn.metrics import roc_auc_score
from datetime import datetime, timedelta
import gcsfs
import duckdb as dd
import pickle
import joblib
from typing import Union
import io
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)

# Function

## calculate_gini

In [2]:
def calculate_gini(pd_scores, bad_indicators):
    """
    Calculate Gini coefficient from scores and binary indicators
    
    Parameters:
    pd_scores: array-like of scores/probabilities
    bad_indicators: array-like of binary outcomes (0/1)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays and ensure they're numeric
    pd_scores = np.array(pd_scores, dtype=float)
    bad_indicators = np.array(bad_indicators, dtype=int)
    
    # Check for valid input data
    if len(pd_scores) == 0 or len(bad_indicators) == 0:
        return np.nan
    
    # Check if we have both good and bad cases (needed for ROC AUC)
    if len(np.unique(bad_indicators)) < 2:
        return np.nan
    
    # Calculate AUC using sklearn
    try:
        auc = roc_auc_score(bad_indicators, pd_scores)
        # Calculate Gini from AUC
        gini = 2 * auc - 1
        return gini
    except ValueError:
        return np.nan

## calculate_periodic_gini_prod_ver_trench_dimfact

In [3]:
import pandas as pd
import numpy as np
from datetime import timedelta
from itertools import combinations

def calculate_gini(scores, labels):
    """
    Calculate Gini coefficient with proper handling of edge cases
    
    Returns np.nan when:
    - Fewer than 2 observations
    - No positive labels (sum of labels = 0)
    """
    n = len(scores)
    if n < 2:
        return np.nan
    
    label_sum = np.sum(labels)
    
    # Handle case where no positive labels exist (all zeros)
    # This prevents division by zero warning
    if label_sum == 0:
        return np.nan
    
    sorted_indices = np.argsort(scores)
    sorted_labels = labels.iloc[sorted_indices].values
    cumsum_labels = np.cumsum(sorted_labels)
    
    gini = 1 - 2 * np.sum(cumsum_labels) / (n * label_sum)
    return gini


def calculate_periodic_gini_prod_ver_trench_dimfact(df, score_column, label_column, namecolumn, 
                                        data_selection_column=None,
                                        model_version_column=None, trench_column=None, 
                                        loan_type_column=None, loan_product_type_column=None,
                                        ostype_column=None,
                                        account_id_column=None):
    """
    Calculate periodic Gini coefficients and return Power BI-friendly long format
    with fact and dimension tables.
    
    Returns:
    - fact_table: Long format with one row per segment per period
    - dimension_table: Unique segment combinations for filtering
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    namecolumn: name for the bad rate label
    data_selection_column: (optional) name of column for data selection (Test/Train)
    model_version_column: (optional) name of column for model version
    trench_column: (optional) name of column for trench category
    loan_type_column: (optional) name of loan type column
    loan_product_type_column: (optional) name of loan product type column
    ostype_column: (optional) name of column for OS type
    account_id_column: (optional) name of column for distinct account IDs
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
    
    optional_columns = {
        'data_selection': data_selection_column,
        'model_version': model_version_column,
        'trench': trench_column,
        'loan_type': loan_type_column,
        'loan_product_type': loan_product_type_column,
        'ostype': ostype_column,
        'account_id': account_id_column
    }
    
    for col_name, col in optional_columns.items():
        if col and col not in df.columns:
            raise ValueError(f"{col_name.replace('_', ' ').title()} column '{col}' not found in dataframe")
    
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Define list of datasets to process
    datasets_to_process = [('Overall', df, {})]
    
    # Create list of available segment columns
    segment_columns = []
    if data_selection_column:
        segment_columns.append(('DataSelection', data_selection_column))
    if model_version_column:
        segment_columns.append(('ModelVersion', model_version_column))
    if trench_column:
        segment_columns.append(('Trench', trench_column))
    if loan_type_column:
        segment_columns.append(('LoanType', loan_type_column))
    if loan_product_type_column:
        segment_columns.append(('ProductType', loan_product_type_column))
    if ostype_column:
        segment_columns.append(('OSType', ostype_column))
    
    # Generate all possible combinations of segment columns
    for r in range(1, len(segment_columns) + 1):
        for combo in combinations(segment_columns, r):
            def generate_combinations(df, segment_columns, index=0, current_filter=None, current_name=''):
                if current_filter is None:
                    current_filter = {}
                
                if index >= len(segment_columns):
                    filtered_df = df
                    for col, val in current_filter.items():
                        filtered_df = filtered_df[filtered_df[col] == val]
                    
                    if len(filtered_df) > 0:
                        yield (current_name.strip('_'), filtered_df, current_filter.copy())
                    return
                
                seg_name, seg_col = segment_columns[index]
                for seg_value in sorted(df[seg_col].dropna().unique()):
                    new_filter = current_filter.copy()
                    new_filter[seg_col] = seg_value
                    new_name = current_name + f'{seg_name}_{seg_value}_'
                    
                    yield from generate_combinations(df, segment_columns, index + 1, new_filter, new_name)
            
            for combo_name, combo_df, combo_metadata in generate_combinations(df, list(combo)):
                datasets_to_process.append((combo_name, combo_df, combo_metadata))
    
    all_results = []
    
    # Process each dataset
    for dataset_name, dataset_df, metadata in datasets_to_process:
        # Calculate weekly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['week'] = dataset_df_copy['disbursementdate'].dt.to_period('W')
        weekly_gini = dataset_df_copy.groupby('week').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 10 else np.nan
        ).reset_index(name='gini_value')
        weekly_gini['period'] = 'Week'
        weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
        weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
        
        # Add distinct account count for weekly
        if account_id_column:
            weekly_account_counts = dataset_df_copy.groupby('week')[account_id_column].nunique().reset_index()
            weekly_account_counts.columns = ['week', 'distinct_accounts']
            weekly_gini = weekly_gini.merge(weekly_account_counts, on='week', how='left')
        else:
            weekly_gini['distinct_accounts'] = None
        
        weekly_gini = weekly_gini[['start_date', 'end_date', 'gini_value', 'period', 'distinct_accounts']]
        
        # Calculate monthly Gini
        dataset_df_copy = dataset_df.copy()
        dataset_df_copy['month'] = dataset_df_copy['disbursementdate'].dt.to_period('M')
        monthly_gini = dataset_df_copy.groupby('month').apply(
            lambda x: calculate_gini(x[score_column], x[label_column])
            if len(x) >= 20 else np.nan
        ).reset_index(name='gini_value')
        monthly_gini['period'] = 'Month'
        monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
        monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
        
        # Add distinct account count for monthly
        if account_id_column:
            monthly_account_counts = dataset_df_copy.groupby('month')[account_id_column].nunique().reset_index()
            monthly_account_counts.columns = ['month', 'distinct_accounts']
            monthly_gini = monthly_gini.merge(monthly_account_counts, on='month', how='left')
        else:
            monthly_gini['distinct_accounts'] = None
        
        monthly_gini = monthly_gini[['start_date', 'end_date', 'gini_value', 'period', 'distinct_accounts']]
        
        # Combine results for this dataset
        gini_results = pd.concat([weekly_gini, monthly_gini], ignore_index=True)
        gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
        
        # Add metadata columns
        gini_results['Model_Name'] = score_column
        gini_results['bad_rate'] = namecolumn
        gini_results['segment_type'] = dataset_name
        
        # Add individual segment components
        gini_results['data_selection'] = metadata.get(data_selection_column, None) if data_selection_column else None
        gini_results['model_version'] = metadata.get(model_version_column, None) if model_version_column else None
        gini_results['trench_category'] = metadata.get(trench_column, None) if trench_column else None
        gini_results['loan_type'] = metadata.get(loan_type_column, None) if loan_type_column else None
        gini_results['loan_product_type'] = metadata.get(loan_product_type_column, None) if loan_product_type_column else None
        gini_results['ostype'] = metadata.get(ostype_column, None) if ostype_column else None
        
        all_results.append(gini_results)
    
    # Combine all results
    fact_table = pd.concat(all_results, ignore_index=True)
    
    # Create dimension table (unique segment combinations for filtering)
    dimension_table = fact_table[['Model_Name', 'bad_rate', 'segment_type', 'data_selection', 'model_version', 
                                   'trench_category', 'loan_type', 'loan_product_type', 'ostype']].drop_duplicates().reset_index(drop=True)
    dimension_table['segment_id'] = range(len(dimension_table))
    
    # Add segment_id to fact table
    fact_table = fact_table.merge(dimension_table[['segment_id', 'Model_Name', 'bad_rate', 'segment_type', 
                                                     'data_selection', 'model_version', 'trench_category', 'loan_type', 'loan_product_type', 'ostype']], 
                                  on=['Model_Name', 'bad_rate', 'segment_type', 'data_selection', 'model_version', 
                                      'trench_category', 'loan_type', 'loan_product_type', 'ostype'], 
                                  how='left')
    
    # Reorder columns in fact table
    fact_table = fact_table[['segment_id', 'start_date', 'end_date', 'period', 'gini_value', 'distinct_accounts',
                             'Model_Name', 'bad_rate', 'segment_type', 'data_selection', 'model_version', 'trench_category', 
                             'loan_type', 'loan_product_type', 'ostype']]
    
    # Reorder columns in dimension table
    dimension_table = dimension_table[['segment_id', 'Model_Name', 'bad_rate', 'segment_type', 'data_selection', 
                                        'model_version', 'trench_category', 'loan_type', 'loan_product_type', 'ostype']]
    
    return fact_table, dimension_table


# Usage:
# fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
#     df_concat, 
#     'Alpha_cic_sil_score', 
#     'deffpd0', 
#     'FPD0',
#     data_selection_column='Data_selection',
#     model_version_column='modelVersionId',
#     trench_column='trenchCategory',
#     loan_type_column='loan_type',
#     loan_product_type_column='loan_product_type',
#     ostype_column='osType',
#     account_id_column='digitalLoanAccountId'
# )
# 
# # In Power BI:
# # 1. Import fact_table and dimension_table
# # 2. Create relationship: dimension_table[segment_id] -> fact_table[segment_id]
# # 3. Use dimension table columns as filters (including ostype)
# # 4. Create DAX measures:
# #    - Gini Measure = AVERAGE(fact_table[gini_value])
# #    - Account Count = SUM(fact_table[distinct_accounts])
# # 5. Use start_date, end_date, period for time-based analysis

# update_tables

In [4]:

def update_tables(fact_table: pd.DataFrame, dimension_table: pd.DataFrame, model_name: str, product: str) -> tuple:
    """
    Updates fact_table and dimension_table:
    - Sets 'Model_display_name' to the given model_name
    - Replaces NaN values in specified columns with 'Overall'
    
    Returns:
        Updated fact_table and dimension_table as a tuple
    """
    # Columns where missing values should be replaced
    cols_to_replace = ['model_version', 'trench_category', 'loan_type', 'loan_product_type', 'ostype']
    
    # Update fact_table
    fact_table['Model_display_name'] = model_name
    fact_table['Product_Category'] = product
    fact_table[cols_to_replace] = fact_table[cols_to_replace].fillna('Overall')
    
    # Update dimension_table
    dimension_table['Model_display_name'] = model_name
    dimension_table['Product_Category'] = product
    dimension_table[cols_to_replace] = dimension_table[cols_to_replace].fillna('Overall')
    
    return fact_table, dimension_table


In [5]:
facttable_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
dimtable_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"

## Beta-Cash-Stack-Model- Credo Score 

### FPD0 

### Test 

In [6]:

sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    deviceOs osType,
    FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
       
    coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and flg_mature_fpd0 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;



"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (20326, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2812644,1f65ba22-02ce-4ed1-969a-1c68a2db6ee5,60828126440025,beta_stack_model_cash,0.0712,"{""demo_score"": ""0.5906670956920956"", ""credo_sc...",2025-11-07 23:21:00,2025-11-09,2025-11,Prod,0,1,Quick,v1,Trench 2,not applicable,android
1,3735926,5f9379d7-5442-45ce-b021-e4119e686218,60837359260017,beta_stack_model_cash,0.3554,"{""demo_score"": ""0.4752730550913879"", ""credo_sc...",2025-10-10 20:06:11,2025-10-11,2025-10,Prod,0,1,Quick,v1,Trench 1,not applicable,android
2,3539548,9cfd1d96-ebef-4c6b-aa1c-d0858d564b8d,60835395480023,beta_stack_model_cash,0.1304,"{""demo_score"": ""0.3310962124306496"", ""credo_sc...",2025-10-10 23:58:55,2025-10-11,2025-10,Prod,0,1,Quick,v1,Trench 3,not applicable,android
3,2433929,c34b5171-223b-4c16-9e28-3464fddc083e,60824339290017,beta_stack_model_cash,0.0924,"{""demo_score"": ""0.5036502546028016"", ""credo_sc...",2025-11-09 09:38:34,2025-11-09,2025-11,Prod,0,1,Quick,v1,Trench 2,not applicable,android
4,3959777,ebc1198f-5303-4186-abfc-45bd65946a53,60839597770011,beta_stack_model_cash,0.3972,"{""demo_score"": ""0.49719744385198683"", ""credo_s...",2026-01-01 18:57:55,2026-01-01,2026-01,Prod,0,1,Quick,v1.1,Trench 1,not applicable,android


In [7]:
df1 = dfd.copy()

### Train 

In [8]:
sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    Data_selection, deviceOs osType,
    FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
  deffpd0,
  flg_mature_fpd0,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
       
    coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and flg_mature_fpd0 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (54566, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2403774,0504c8a6-bf56-4566-bb3b-c4211b79884f,60824037740029,Beta-Cash-Stack-Model,0.120978,"{""demo_score"": 0.47289208952730427, ""credo_sco...",2024-10-15 13:52:28,2024-10-15,2024-10,Dev_Train,1,1,Quick,v1,Trench 3,not applicable,ios
1,2461482,064f08e4-be8c-4ecb-8fc1-7ff842546931,60824614820014,Beta-Cash-Stack-Model,0.088297,"{""demo_score"": 0.3327346533252929, ""apps_score...",2024-10-14 11:06:44,2024-10-15,2024-10,Dev_Train,0,1,Quick,v1,Trench 2,not applicable,android
2,2333952,06935c14-06b7-4423-a217-9d63ccee3fa3,60823339520028,Beta-Cash-Stack-Model,0.164747,"{""demo_score"": 0.577890529092715, ""credo_score...",2024-10-23 05:46:23,2024-10-23,2024-10,Dev_Train,1,1,Quick,v1,Trench 3,not applicable,ios
3,2922739,091ba8a0-f332-4226-82c9-fd38baff1c66,60829227390014,Beta-Cash-Stack-Model,0.238052,"{""demo_score"": 0.46777097317923905, ""apps_scor...",2024-10-09 13:43:06,2024-10-09,2024-10,Dev_Train,0,1,Quick,v1.1,Trench 1,not applicable,android
4,2909216,09391578-89f4-44e5-a63c-62d176296d3a,60829092160011,Beta-Cash-Stack-Model,0.176933,"{""demo_score"": 0.4821367398860277, ""apps_score...",2024-10-04 12:53:08,2024-10-04,2024-10,Dev_Train,0,1,Quick,v1,Trench 1,not applicable,android


In [9]:
df2 = dfd.copy()

### Concat 

In [10]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (69161, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd0,flg_mature_fpd0,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3539548,9cfd1d96-ebef-4c6b-aa1c-d0858d564b8d,60835395480023,beta_stack_model_cash,0.1304,"{""demo_score"": ""0.3310962124306496"", ""credo_sc...",2025-10-10 23:58:55,2025-10-11,2025-10,Prod,0,1,Quick,v1,Trench 3,not applicable,android
1,3959777,ebc1198f-5303-4186-abfc-45bd65946a53,60839597770011,beta_stack_model_cash,0.3972,"{""demo_score"": ""0.49719744385198683"", ""credo_s...",2026-01-01 18:57:55,2026-01-01,2026-01,Prod,0,1,Quick,v1.1,Trench 1,not applicable,android
2,1084210,22132ac5-6700-4b3f-ae32-501fdaba075e,60810842100013,beta_stack_model_cash,0.1621,"{""demo_score"": ""0.4547455324627957"", ""credo_sc...",2026-01-01 18:58:56,2026-01-01,2026-01,Prod,0,1,Quick,v1.1,Trench 2,not applicable,ios
3,1080796,27ed2366-6b6c-4c53-b984-ef57da6466bd,60810807960064,beta_stack_model_cash,0.1062,"{""demo_score"": ""0.31458623119673257"", ""credo_s...",2025-10-02 01:05:15,2025-10-02,2025-10,Prod,0,1,Quick,v1,Trench 3,not applicable,ios
4,3712273,683f59e0-f765-4fa8-94aa-aa638c9ef18d,60837122730011,beta_stack_model_cash,0.1958,"{""demo_score"": ""0.5521387196761803"", ""credo_sc...",2025-09-29 11:07:24,2025-09-29,2025-09,Prod,1,1,Quick,v1,Trench 1,not applicable,ios


In [11]:
# df_concat = df1.copy()

df_concat['credo_score'] = pd.to_numeric(df_concat['credo_score'], errors='coerce')

In [12]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'credo_score', 
    'deffpd0', 
    'FPD0',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    ostype_column='osType', 
    account_id_column='digitalLoanAccountId'
)

In [13]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_stack_credo_score_cash', product='CASH')
print(f"The shape of the fact table is:\t {fact_table.shape}")
print(f"The shape of the dimension table is:\t {dimension_table.shape}")

The shape of the fact table is:	 (21676, 17)
The shape of the dimension table is:	 (504, 12)


In [14]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d7dad59c-2326-42ff-9966-31f4ebd1da34>

In [15]:
# Upload to BigQuery

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0c0ca00f-c420-463d-96e7-7bc3f41bda3f>

### FPD10 

### Test 

In [16]:

sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
    deviceOs osType,
    FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and del.flg_mature_fpd10 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;



"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (17478, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3924492,94008fd7-dac0-4da9-b5a8-9a7a8620280e,60839244920015,beta_stack_model_cash,0.1721,"{""demo_score"": ""0.598889517865591"", ""credo_sco...",2025-12-22 16:28:15,2025-12-22,2025-12,Prod,0,1,Quick,v1.1,Trench 1,not applicable,android
1,3545985,a752bc9a-bbd8-4c55-8f96-da0ba9151d4d,60835459850011,beta_stack_model_cash,0.1937,"{""demo_score"": ""0.5254480873977354"", ""credo_sc...",2025-10-03 06:57:03,2025-10-04,2025-10,Prod,0,1,Quick,v1,Trench 2,not applicable,ios
2,3736169,af604ccd-48d7-439e-81c7-c11944afa29f,60837361690018,beta_stack_model_cash,0.5177,"{""demo_score"": ""0.5351002076883583"", ""credo_sc...",2025-10-11 01:02:01,2025-10-11,2025-10,Prod,0,1,Quick,v1,Trench 1,not applicable,android
3,3734595,cc50c2a7-1f2a-4d8f-be52-0826fc55e26d,60837345950014,beta_stack_model_cash,0.1899,"{""demo_score"": ""0.49096332735511866"", ""credo_s...",2025-10-10 09:38:05,2025-10-11,2025-10,Prod,0,1,Quick,v1,Trench 1,not applicable,android
4,3738046,d74b5189-3508-4804-a889-8b7025b59917,60837380460016,beta_stack_model_cash,0.1664,"{""demo_score"": ""0.4639746786139382"", ""credo_sc...",2025-10-11 20:34:31,2025-10-11,2025-10,Prod,0,1,Quick,v1,Trench 1,not applicable,ios


In [17]:
df1 = dfd.copy()

### Train 

In [18]:
sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature, deviceOs osType,
    Data_selection
    FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
  del.deffpd10,
  del.flg_mature_fpd10,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and del.flg_mature_fpd10 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (54566, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,2942334,1ed885d9-0078-4202-bed5-94cee99fd258,60829423340017,Beta-Cash-Stack-Model,0.363612,"{""demo_score"": 0.577259812525358, ""apps_score""...",2024-10-15 14:21:42,2024-10-15,2024-10,Dev_Train,0,1,Quick,v1,Trench 1,not applicable,android
1,2930006,24c90597-d8ac-478f-ab13-f525513cb4d7,60829300060011,Beta-Cash-Stack-Model,0.076959,"{""demo_score"": 0.4004971906631461, ""apps_score...",2024-10-11 11:00:25,2024-10-11,2024-10,Dev_Train,0,1,Quick,v1.1,Trench 1,not applicable,android
2,2589535,2d1df293-25cd-4f7d-b2a8-e61a25316b3e,60825895350018,Beta-Cash-Stack-Model,0.098301,"{""demo_score"": 0.5417038531824688, ""apps_score...",2024-10-14 17:46:46,2024-10-18,2024-10,Dev_Train,0,1,Quick,v1,Trench 2,not applicable,android
3,2626634,387995f8-33a6-411b-b04f-e9c2e2b63eb5,60826266340014,Beta-Cash-Stack-Model,0.078526,"{""demo_score"": 0.40885540430960127, ""apps_scor...",2024-10-01 23:50:07,2024-10-02,2024-10,Dev_Train,0,1,Quick,v1,Trench 2,not applicable,android
4,2902077,3c7441aa-03b8-46e5-b792-b53d38cfbaaa,60829020770019,Beta-Cash-Stack-Model,0.188862,"{""demo_score"": 0.4199346580925607, ""apps_score...",2024-10-02 01:56:40,2024-10-02,2024-10,Dev_Train,0,1,Quick,v1,Trench 1,not applicable,android


In [19]:
df2 = dfd.copy()

### Concat 

In [20]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (66313, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd10,flg_mature_fpd10,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3924492,94008fd7-dac0-4da9-b5a8-9a7a8620280e,60839244920015,beta_stack_model_cash,0.1721,"{""demo_score"": ""0.598889517865591"", ""credo_sco...",2025-12-22 16:28:15,2025-12-22,2025-12,Prod,0,1,Quick,v1.1,Trench 1,not applicable,android
1,3545985,a752bc9a-bbd8-4c55-8f96-da0ba9151d4d,60835459850011,beta_stack_model_cash,0.1937,"{""demo_score"": ""0.5254480873977354"", ""credo_sc...",2025-10-03 06:57:03,2025-10-04,2025-10,Prod,0,1,Quick,v1,Trench 2,not applicable,ios
2,3738046,d74b5189-3508-4804-a889-8b7025b59917,60837380460016,beta_stack_model_cash,0.1664,"{""demo_score"": ""0.4639746786139382"", ""credo_sc...",2025-10-11 20:34:31,2025-10-11,2025-10,Prod,0,1,Quick,v1,Trench 1,not applicable,ios
3,3508654,00427931-a96e-4fbf-b5dd-4b1c5e2480af,60835086540017,beta_stack_model_cash,0.1404,"{""demo_score"": ""0.46422534108374164"", ""credo_s...",2025-12-23 19:07:37,2025-12-23,2025-12,Prod,0,1,Quick,v1.1,Trench 2,not applicable,ios
4,2064618,394fe099-95ea-4c52-8ef4-43bf8f3f1fc4,60820646180028,beta_stack_model_cash,0.1022,"{""demo_score"": ""0.4252731430511041"", ""credo_sc...",2025-09-28 04:48:13,2025-10-04,2025-09,Prod,0,1,Quick,v1,Trench 3,not applicable,ios


In [21]:
# df_concat = df1.copy()

df_concat['credo_score'] = pd.to_numeric(df_concat['credo_score'], errors='coerce')

In [22]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'credo_score', 
    'deffpd10', 
    'FPD10',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    ostype_column='osType', 
    account_id_column='digitalLoanAccountId'
)

In [23]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_stack_credo_score_cash', product='CASH')
print(f"The shape of the fact table is:\t {fact_table.shape}")
print(f"The shape of the dimension table is:\t {dimension_table.shape}")

The shape of the fact table is:	 (21388, 17)
The shape of the dimension table is:	 (504, 12)


In [24]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=abdc6d05-6092-4a0c-be60-d53fa959ccd0>

In [25]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=024be611-5301-4442-bb9d-3a1705890368>

### FPD30 

### Test 

In [26]:

sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,  deviceOs osType,
    FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and del.flg_mature_fpd30 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;



"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (11037, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3719077,70b0e190-3125-4e0a-acab-ce37fa7ea02a,60837190770017,beta_stack_model_cash,0.2063,"{""demo_score"": ""0.45830241485823703"", ""credo_s...",2025-10-02 16:31:08,2025-10-02,2025-10,Prod,0,1,Quick,v1,Trench 1,not applicable,ios
1,2098110,a6161ce3-1a71-4da0-b435-59a575c40038,60820981100034,beta_stack_model_cash,0.0961,"{""demo_score"": ""0.24815123031747535"", ""credo_s...",2025-09-30 23:03:59,2025-10-01,2025-09,Prod,0,1,Quick,v1,Trench 3,not applicable,android
2,3446950,4469e403-cd85-4853-9627-9ea92e39cdff,60834469500018,beta_stack_model_cash,0.2647,"{""demo_score"": ""0.6001222885315103"", ""credo_sc...",2025-09-27 14:14:11,2025-09-27,2025-09,Prod,0,1,Quick,v1,Trench 2,not applicable,ios
3,3839465,f01f8b31-9e54-4425-ac06-2faaee295300,60838394650011,beta_stack_model_cash,0.0515,"{""demo_score"": ""0.5980025672415492"", ""credo_sc...",2025-11-27 10:21:37,2025-12-04,2025-11,Prod,0,1,Quick,v1,Trench 1,not applicable,android
4,2642293,c81869ae-c522-4839-91a8-b8e7285e3c0f,60826422930033,beta_stack_model_cash,0.1634,"{""demo_score"": ""0.5521915931467914"", ""credo_sc...",2025-09-26 15:49:49,2025-09-26,2025-09,Prod,1,1,Quick,v1,Trench 3,not applicable,android


In [27]:
df1 = dfd.copy()

### Train 

In [28]:
sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,  deviceOs osType,
    Data_selection
    FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
  del.deffpd30,
  del.flg_mature_fpd30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and del.flg_mature_fpd0 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (54566, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,1626424,023ff030-6d99-4987-ba1a-863b854067fc,60816264240019,Beta-Cash-Stack-Model,0.056987,"{""demo_score"": 0.5962616432416837, ""apps_score...",2024-10-27 18:23:37,2024-10-27,2024-10,Dev_Train,0,1,Quick,v1.1,Trench 2,not applicable,android
1,2986424,037482b5-9d19-43aa-be77-6b0957bdc8f2,60829864240014,Beta-Cash-Stack-Model,0.132524,"{""demo_score"": 0.4426496605338299, ""apps_score...",2024-10-31 06:31:20,2024-10-31,2024-10,Dev_Train,0,1,Quick,v1.1,Trench 1,not applicable,android
2,2987335,07277445-8a3b-472a-9317-2073ba6d37c3,60829873350018,Beta-Cash-Stack-Model,0.042565,"{""demo_score"": 0.5062085499355357, ""apps_score...",2024-10-31 13:21:38,2024-10-31,2024-10,Dev_Train,0,1,Quick,v1.1,Trench 1,not applicable,android
3,2898704,078f5136-f303-4980-8611-68d482b0b1ca,60828987040016,Beta-Cash-Stack-Model,0.084556,"{""demo_score"": 0.41398789798877894, ""apps_scor...",2024-10-01 09:32:57,2024-10-01,2024-10,Dev_Train,0,1,Quick,v1,Trench 1,not applicable,android
4,2955221,08b01ba9-4178-4775-94a2-f29263c9bbb4,60829552210016,Beta-Cash-Stack-Model,0.195745,"{""demo_score"": 0.4937612870279114, ""credo_scor...",2024-10-19 17:07:47,2024-10-19,2024-10,Dev_Train,0,1,Quick,v1,Trench 1,not applicable,ios


In [29]:
df2 = dfd.copy()

### Concat 

In [30]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (59888, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffpd30,flg_mature_fpd30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3719077,70b0e190-3125-4e0a-acab-ce37fa7ea02a,60837190770017,beta_stack_model_cash,0.2063,"{""demo_score"": ""0.45830241485823703"", ""credo_s...",2025-10-02 16:31:08,2025-10-02,2025-10,Prod,0,1,Quick,v1,Trench 1,not applicable,ios
1,2098110,a6161ce3-1a71-4da0-b435-59a575c40038,60820981100034,beta_stack_model_cash,0.0961,"{""demo_score"": ""0.24815123031747535"", ""credo_s...",2025-09-30 23:03:59,2025-10-01,2025-09,Prod,0,1,Quick,v1,Trench 3,not applicable,android
2,3446950,4469e403-cd85-4853-9627-9ea92e39cdff,60834469500018,beta_stack_model_cash,0.2647,"{""demo_score"": ""0.6001222885315103"", ""credo_sc...",2025-09-27 14:14:11,2025-09-27,2025-09,Prod,0,1,Quick,v1,Trench 2,not applicable,ios
3,3839465,f01f8b31-9e54-4425-ac06-2faaee295300,60838394650011,beta_stack_model_cash,0.0515,"{""demo_score"": ""0.5980025672415492"", ""credo_sc...",2025-11-27 10:21:37,2025-12-04,2025-11,Prod,0,1,Quick,v1,Trench 1,not applicable,android
4,2642293,c81869ae-c522-4839-91a8-b8e7285e3c0f,60826422930033,beta_stack_model_cash,0.1634,"{""demo_score"": ""0.5521915931467914"", ""credo_sc...",2025-09-26 15:49:49,2025-09-26,2025-09,Prod,1,1,Quick,v1,Trench 3,not applicable,android


In [31]:
# df_concat = df1.copy()

df_concat['credo_score'] = pd.to_numeric(df_concat['credo_score'], errors='coerce')

In [32]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'credo_score', 
    'deffpd30', 
    'FPD30',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    ostype_column='osType', 
    account_id_column='digitalLoanAccountId'
)

In [33]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_stack_credo_score_cash', product='CASH')
print(f"The shape of the fact table is:\t {fact_table.shape}")
print(f"The shape of the dimension table is:\t {dimension_table.shape}")

The shape of the fact table is:	 (20180, 17)
The shape of the dimension table is:	 (456, 12)


In [34]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6b287016-1830-4286-b2e4-0e5df88886d5>

In [35]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=feef3a5c-e8ab-4628-8320-857aee3b7ebe>

### FSPD30 

### Test 

In [36]:

sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
        deviceOs osType,
    FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and del.flg_mature_fspd_30 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;



"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (3711, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3487395,50d4bb08-2124-407f-8bb3-067d22ab1cfe,60834873950017,beta_stack_model_cash,0.1278,"{""demo_score"": ""0.4296130309301335"", ""credo_sc...",2025-09-29 15:29:48,2025-10-03,2025-09,Prod,0,1,Quick,v1,Trench 2,not applicable,ios
1,3721729,4a79f936-068e-4436-8c0c-c78982598fa4,60837217290013,beta_stack_model_cash,0.1896,"{""demo_score"": ""0.4915565700163809"", ""credo_sc...",2025-10-04 08:37:07,2025-10-04,2025-10,Prod,0,1,Quick,v1,Trench 1,not applicable,ios
2,3785229,57d911fa-d7c1-48d6-aa1d-800c2150bf8b,60837852290014,beta_stack_model_cash,0.2173,"{""demo_score"": ""0.4062334915518481"", ""credo_sc...",2025-11-03 11:21:01,2025-11-04,2025-11,Prod,0,1,Quick,v1,Trench 1,not applicable,android
3,1346048,896716c5-e4e5-4b3c-9218-0b605d293c72,60813460480017,beta_stack_model_cash,0.1838,"{""demo_score"": ""0.5258142934829556"", ""credo_sc...",2025-09-25 11:37:29,2025-09-25,2025-09,Prod,0,1,Quick,v1,Trench 2,not applicable,android
4,3726073,b6e2666e-d8d8-409e-8315-907d922e8d23,60837260730016,beta_stack_model_cash,0.1307,"{""demo_score"": ""0.461432292509667"", ""credo_sco...",2025-10-05 20:20:43,2025-10-06,2025-10,Prod,0,1,Quick,v1,Trench 1,not applicable,ios


In [37]:
df1 = dfd.copy()

### Train 

In [38]:
sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,     deviceOs osType,
    Data_selection
    FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  Data_selection,  
  del.deffspd30,
  del.flg_mature_fspd_30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and del.flg_mature_fspd_30 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (50724, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3783460,6f8dd626-71a8-48d2-b578-2e5efe08ebbf,60837834600011,Beta-Cash-Stack-Model,0.0797,"{""demo_score"": 0.4312799873202566, ""apps_score...",2025-11-03 09:55:28,2025-11-03,2025-11,Dev_Test,0,1,Quick,v1.1,Trench 1,not applicable,android
1,3785283,cd6949d4-66ae-4950-8cc3-ba05e99c773f,60837852830011,Beta-Cash-Stack-Model,0.2076,"{""demo_score"": 0.5991159938298594, ""apps_score...",2025-11-01 22:16:03,2025-11-02,2025-11,Dev_Test,0,1,Quick,v1.1,Trench 1,not applicable,android
2,3786687,48b70bb5-66a3-42bd-9aa9-ea41b675ddff,60837866870011,Beta-Cash-Stack-Model,0.0771,"{""demo_score"": 0.5169548363332349, ""apps_score...",2025-11-02 14:56:14,2025-11-02,2025-11,Dev_Test,0,1,Quick,v1.1,Trench 1,not applicable,android
3,1036751,6a1b17d6-2cbd-438f-91a8-f4d4e49bf0b2,60810367510036,Beta-Cash-Stack-Model,0.2210,"{""demo_score"": 0.5969671334800911, ""apps_score...",2025-11-02 13:14:44,2025-11-04,2025-11,Dev_Test,1,1,Quick,v1.1,Trench 2,not applicable,android
4,3785008,9b0f3373-3201-46e6-906e-959e132d4727,60837850080013,Beta-Cash-Stack-Model,0.1385,"{""demo_score"": 0.5833512834987409, ""apps_score...",2025-11-01 18:35:43,2025-11-01,2025-11,Dev_Test,0,1,Quick,v1.1,Trench 1,not applicable,android


In [39]:
df2 = dfd.copy()

### Concat 

In [40]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (52483, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffspd30,flg_mature_fspd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3487395,50d4bb08-2124-407f-8bb3-067d22ab1cfe,60834873950017,beta_stack_model_cash,0.1278,"{""demo_score"": ""0.4296130309301335"", ""credo_sc...",2025-09-29 15:29:48,2025-10-03,2025-09,Prod,0,1,Quick,v1,Trench 2,not applicable,ios
1,3721729,4a79f936-068e-4436-8c0c-c78982598fa4,60837217290013,beta_stack_model_cash,0.1896,"{""demo_score"": ""0.4915565700163809"", ""credo_sc...",2025-10-04 08:37:07,2025-10-04,2025-10,Prod,0,1,Quick,v1,Trench 1,not applicable,ios
2,3726073,b6e2666e-d8d8-409e-8315-907d922e8d23,60837260730016,beta_stack_model_cash,0.1307,"{""demo_score"": ""0.461432292509667"", ""credo_sco...",2025-10-05 20:20:43,2025-10-06,2025-10,Prod,0,1,Quick,v1,Trench 1,not applicable,ios
3,2193432,154f68d6-e38e-4e17-bb35-a9d61de32168,60821934320027,beta_stack_model_cash,0.1175,"{""demo_score"": ""0.5663950467342673"", ""credo_sc...",2025-09-26 22:01:48,2025-09-27,2025-09,Prod,1,1,Quick,v1,Trench 3,not applicable,ios
4,3712884,6f477abe-990f-4931-b210-90f40be222ca,60837128840012,beta_stack_model_cash,0.1506,"{""demo_score"": ""0.3852775483492032"", ""credo_sc...",2025-10-03 17:30:51,2025-10-03,2025-10,Prod,0,1,Quick,v1,Trench 1,not applicable,ios


In [41]:
# df_concat = df1.copy()

df_concat['credo_score'] = pd.to_numeric(df_concat['credo_score'], errors='coerce')

In [42]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'credo_score', 
    'deffspd30', 
    'FSPD30',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
    ostype_column='osType',  
    account_id_column='digitalLoanAccountId'
)

In [43]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_stack_credo_score_cash', product='CASH')
print(f"The shape of the fact table is:\t {fact_table.shape}")
print(f"The shape of the dimension table is:\t {dimension_table.shape}")

The shape of the fact table is:	 (18744, 17)
The shape of the dimension table is:	 (456, 12)


In [44]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=645c20d8-8291-43f8-9826-972a1f0e885d>

In [45]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2a6dc124-7bd9-4f68-864f-dac935482260>

### FSTPD30 

### Test 

In [46]:

sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,
     deviceOs osType,
    FROM prj-prod-dataplatform.audit_balance.ml_model_run_details mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
  'Prod' Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and del.flg_mature_fstpd_30 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;



"""
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()



The shape of the dataframe downloaded is:	 (73, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,1080796,27ed2366-6b6c-4c53-b984-ef57da6466bd,60810807960064,beta_stack_model_cash,0.1062,"{""demo_score"": ""0.31458623119673257"", ""credo_s...",2025-10-02 01:05:15,2025-10-02,2025-10,Prod,0,1,Quick,v1,Trench 3,not applicable,ios
1,3712273,683f59e0-f765-4fa8-94aa-aa638c9ef18d,60837122730011,beta_stack_model_cash,0.1958,"{""demo_score"": ""0.5521387196761803"", ""credo_sc...",2025-09-29 11:07:24,2025-09-29,2025-09,Prod,0,1,Quick,v1,Trench 1,not applicable,ios
2,3709389,c7182945-70ff-4091-bccd-47241b8fce48,60837093890015,beta_stack_model_cash,0.2764,"{""demo_score"": ""0.5338976464327317"", ""credo_sc...",2025-09-27 23:18:25,2025-09-29,2025-09,Prod,0,1,Quick,v1,Trench 1,not applicable,android
3,2642293,c81869ae-c522-4839-91a8-b8e7285e3c0f,60826422930033,beta_stack_model_cash,0.1634,"{""demo_score"": ""0.5521915931467914"", ""credo_sc...",2025-09-26 15:49:49,2025-09-26,2025-09,Prod,1,1,Quick,v1,Trench 3,not applicable,android
4,2972716,f3ddbb3e-afea-4f96-861b-768c1abb7e5b,60829727160012,beta_stack_model_cash,0.3094,"{""demo_score"": ""0.5450310640829273"", ""credo_sc...",2025-09-29 09:20:58,2025-09-29,2025-09,Prod,0,1,Quick,v1,Trench 2,not applicable,android


In [47]:
df1 = dfd.copy()

### Train 

In [48]:
sq = """
with modelname as 
  (  SELECT
    mmrd.customerId,mmrd.digitalLoanAccountId,prediction,start_time,end_time,modelDisplayName,modelVersionId, 
  case when trenchCategory is null then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench1' end) 
     when trenchCategory = '' then (case when mt.ln_user_type='1_Repeat Applicant' then 'Trench 3'
    when mt.ln_user_type <>'1_Repeat Applicant' and DATE_DIFF(current_date(), mt.onb_tsa_onboarding_datetime, DAY) >30 then 'Trench 2'
    else 'Trench 1' end) 
    else trenchCategory end  as trenchCategory,
    REPLACE(REPLACE(calcFeature, "'", '"'), "None", "null") AS calcFeature,  deviceOs osType,
    Data_selection
    FROM prj-prod-dataplatform.dap_ds_poweruser_playground.ml_training_model_run_details_20260116 mmrd
  left join prj-prod-dataplatform.risk_credit_mis.model_loan_score_mart mt on mt.digitalLoanAccountId = mmrd.digitalLoanAccountId
  WHERE modelDisplayName in ('Beta-Cash-Stack-Model', 'beta_stack_model_cash')
  -- and modelVersionId = 'v1'
      ),
  deliquency as
(select loanAccountNumber,
case when obs_min_inst_def0 >= 1 and min_inst_def0 = 1 then 1 else 0 end deffpd0,
case when obs_min_inst_def10 >=1 and min_inst_def10 =1 then 1 else 0 end deffpd10,
case when obs_min_inst_def30 >=1 and min_inst_def30 =1 then 1 else 0 end deffpd30,
case when obs_min_inst_def30 >=2 and min_inst_def30 in (1,2) then 1 else 0 end deffspd30,
case when obs_min_inst_def30 >=3 and min_inst_def30 in (1,2,3) then 1 else 0 end deffstpd30,
case when obs_min_inst_def0 >= 1 then 1 else 0 end flg_mature_fpd0,
case when obs_min_inst_def10 >=1 then 1 else 0 end flg_mature_fpd10,
case when obs_min_inst_def30 >=1 then 1 else 0 end flg_mature_fpd30,
case when obs_min_inst_def30 >=2 then 1 else 0 end flg_mature_fspd_30,
case when obs_min_inst_def30 >=3 then 1 else 0 end flg_mature_fstpd_30
from prj-prod-dataplatform.risk_credit_mis.loan_deliquency_data),
base as 
  (select distinct r.customerId,
  r.digitalLoanAccountId,
  loanmaster.loanAccountNumber,
  r.modelDisplayName,
  coalesce(cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64), cast(JSON_VALUE(SAFE.PARSE_JSON(CAST(calcFeature AS STRING)), '$.credo_score')AS FLOAT64)) AS credo_score,
  calcFeature,
  coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime)) AS appln_submit_datetime,
  date(loanmaster.disbursementDateTime) disbursementdate,
  format_date('%Y-%m', coalesce(IF(loanmaster.new_loan_type = 'Flex-up', loanmaster.startApplyDateTime, loanmaster.termsAndConditionsSubmitDateTime),  cast(r.start_time as datetime))) as Application_month, 
   Data_selection,  
  del.deffstpd30,
  del.flg_mature_fstpd_30,
  loanmaster.new_loan_type,
  modelVersionId, trenchCategory,
    case when loanmaster.loantype='BNPL' and store_type =1 then 'Appliance'
    when loanmaster.loantype='BNPL' and store_type =2 then 'Mobile' 
    when loanmaster.loantype='BNPL' and store_type =3 then 'Mall' 
    when loanmaster.loantype='BNPL' and store_type not in (1,2,3) then store_tagging
    else 'not applicable' end as loan_product_type,
        coalesce((case when lower(r.osType) like '%andro%' then 'android'
                  when lower(r.osType) like '%os%' then 'ios' else lower(r.osType) end), 
            (case when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%andro%' then 'android'
                  when lower(coalesce(loanmaster.osversion_v2, loanmaster.osVersion)) like '%os%' then 'ios'
                  when lower(loanmaster.deviceType) like '%andro%' then 'android'
                  else 'ios' end)
            ) as osType
  from modelname r
  left join risk_credit_mis.loan_master_table loanmaster  ON loanmaster.digitalLoanAccountId = r.digitalLoanAccountId
  left join deliquency del on del.loanAccountNumber = loanmaster.loanAccountNumber
   left join(SELECT DISTINCT mer_refferal_code, mer_name mer_name,store_type,store_tagging FROM `dl_loans_db_raw.tdbk_merchant_refferal_mtb`
  left join worktable_datachampions.TARGET_SPLIT P on P.STORE_NAME = mer_name
 qualify row_number() over(partition by mer_refferal_code order by  created_dt desc)=1) sil_category on loanmaster.purpleKey=sil_category.mer_refferal_code
  where 
  loanmaster.flagDisbursement = 1
  and loanmaster.disbursementDateTime is not null
  and flg_mature_fstpd_30 = 1
  )
  select *
  from base
  where credo_score is not null
 qualify row_number() over(partition by digitalLoanAccountId, modelVersionId order by appln_submit_datetime) = 1
   ;
  """
dfd = client.query(sq).to_dataframe()
# dfd = dfd.drop_duplicates(keep='first')
print(f"The shape of the dataframe downloaded is:\t {dfd.shape}")
dfd.head()

The shape of the dataframe downloaded is:	 (48606, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,3715840,d95644bb-839e-4aeb-9ec9-85aff1693679,60837158400021,Beta-Cash-Stack-Model,0.140089,"{""demo_score"": 0.39621899130316895, ""apps_scor...",2025-10-01 01:19:47,2025-10-03,2025-10,Dev_Test,0,1,Quick,v1.1,Trench 1,not applicable,android
1,2825943,c77f5c92-f070-49fa-9e6c-834a6e7dd9b7,60828259430012,Beta-Cash-Stack-Model,0.394200,"{""demo_score"": 0.49042435913862126, ""apps_scor...",2025-10-01 10:37:41,2025-10-01,2025-10,Dev_Test,0,1,Quick,v1.1,Trench 2,not applicable,android
2,3712217,6b85a5fb-2b6a-4b71-afde-838f71e3f3b2,60837122170013,Beta-Cash-Stack-Model,0.124715,"{""demo_score"": 0.5215996837976284, ""apps_score...",2025-10-01 19:16:49,2025-10-03,2025-10,Dev_Test,0,1,Quick,v1.1,Trench 1,not applicable,android
3,2898617,ae1631db-5022-4c4a-95b1-9c1b681e4257,60828986170026,Beta-Cash-Stack-Model,0.052500,"{""demo_score"": 0.49187605740895396, ""apps_scor...",2025-10-02 03:07:32,2025-10-02,2025-10,Dev_Test,0,1,Quick,v1.1,Trench 2,not applicable,android
4,1787644,fb344fc2-5029-4120-9677-0850357aec53,60817876440019,Beta-Cash-Stack-Model,0.303700,"{""demo_score"": 0.49454198043552583, ""apps_scor...",2025-10-02 21:31:25,2025-10-02,2025-10,Dev_Test,0,1,Quick,v1.1,Trench 2,not applicable,android


In [49]:
df2 = dfd.copy()

### Concat 

In [50]:
# df_concat = pd.concat([df1, df2], ignore_index=True)
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")

# df_concat = (df_concat
#              .sort_values(by=['digitalLoanAccountId', 'Data_selection'],
#                           key=lambda s: s.map({'Train': 0, 'Test': 1}))
#              .drop_duplicates(subset=['digitalLoanAccountId'], keep='first')
#              .reset_index(drop=True))
# print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
# df_concat.head()


# 1) Get all IDs present in Train
train_ids = set(df2['digitalLoanAccountId'])

# 2) Keep only Test rows whose ID is NOT in Train
df1_no_dupes = df1[~df1['digitalLoanAccountId'].isin(train_ids)]

# 3) Concatenate
df_concat = pd.concat([df1_no_dupes, df2], ignore_index=True)

print(f"The shape of the concatenated dataframe is:\t {df_concat.shape}")
df_concat.head()


The shape of the concatenated dataframe is:	 (48654, 17)


,customerId,digitalLoanAccountId,loanAccountNumber,modelDisplayName,credo_score,calcFeature,appln_submit_datetime,disbursementdate,Application_month,Data_selection,deffstpd30,flg_mature_fstpd_30,new_loan_type,modelVersionId,trenchCategory,loan_product_type,osType
0,1080796,27ed2366-6b6c-4c53-b984-ef57da6466bd,60810807960064,beta_stack_model_cash,0.1062,"{""demo_score"": ""0.31458623119673257"", ""credo_s...",2025-10-02 01:05:15,2025-10-02,2025-10,Prod,0,1,Quick,v1,Trench 3,not applicable,ios
1,3712273,683f59e0-f765-4fa8-94aa-aa638c9ef18d,60837122730011,beta_stack_model_cash,0.1958,"{""demo_score"": ""0.5521387196761803"", ""credo_sc...",2025-09-29 11:07:24,2025-09-29,2025-09,Prod,0,1,Quick,v1,Trench 1,not applicable,ios
2,2642293,c81869ae-c522-4839-91a8-b8e7285e3c0f,60826422930033,beta_stack_model_cash,0.1634,"{""demo_score"": ""0.5521915931467914"", ""credo_sc...",2025-09-26 15:49:49,2025-09-26,2025-09,Prod,1,1,Quick,v1,Trench 3,not applicable,android
3,3140719,f755aeb4-fbc4-4d90-975d-8980f8da5a1f,60831407190029,beta_stack_model_cash,0.1372,"{""demo_score"": ""0.4136465182333435"", ""credo_sc...",2025-09-27 20:07:11,2025-09-27,2025-09,Prod,0,1,Quick,v1,Trench 3,not applicable,ios
4,3706434,48e3eccc-296c-4696-9d17-bf71a0640d53,60837064340011,beta_stack_model_cash,0.2421,"{""demo_score"": ""0.5740309168811254"", ""credo_sc...",2025-09-28 11:16:08,2025-10-02,2025-09,Prod,0,1,Quick,v1,Trench 1,not applicable,ios


In [51]:
# df_concat = df1.copy()

df_concat['credo_score'] = pd.to_numeric(df_concat['credo_score'], errors='coerce')

In [52]:
fact_table, dimension_table = calculate_periodic_gini_prod_ver_trench_dimfact(
    df_concat, 
    'credo_score', 
    'deffstpd30', 
    'FSTPD30',
    data_selection_column='Data_selection',
    model_version_column='modelVersionId',
    trench_column='trenchCategory',
    loan_type_column='new_loan_type',
    loan_product_type_column='loan_product_type',
     ostype_column='osType',  
    account_id_column='digitalLoanAccountId'
)

In [53]:
fact_table, dimension_table = update_tables(fact_table, dimension_table, model_name='beta_stack_credo_score_cash', product='CASH')
print(f"The shape of the fact table is:\t {fact_table.shape}")
print(f"The shape of the dimension table is:\t {dimension_table.shape}")

The shape of the fact table is:	 (17220, 17)
The shape of the dimension table is:	 (440, 12)


In [54]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.fact_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(fact_table, facttable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=227d4fb9-6a1a-452b-ab95-ceed8c62a70d>

In [55]:
# Upload to BigQuery
# table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.dimension_table3"
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND",  # or "WRITE_APPEND"
)
job = client.load_table_from_dataframe(dimension_table, dimtable_id, job_config=job_config)
job.result()  # Wait for the job to complete

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ddc320c8-a01b-4ab3-9fd6-cbad00082706>